In [2]:
# import all necessary libraries
import time
import sys
import random
import numpy as np
from register.QuantumRegister import *
from register.grovers import *
from register.deutsch import * 
from register.deutschJozsa import * 
import matplotlib.pyplot as plt 
import matplotlib.ticker as mticker
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display
import random
import scipy

In [ ]:
# define the function to fit curve to data
def fitFunc(x, a, b):
    return 2**(a*(x+b))

simRange = 3

In [ ]:

# define the function to test the time taken for the Grover's algorithm

def groversTimeElapsed(n_qubits):

    listLength = int(2**n_qubits)
    groverList = np.zeros(listLength)
    assert n_qubits == int(np.ceil(np.log2(listLength)))

    target = random.randint(0,listLength-1)
    register = QuantumRegister(n_qubits)

    def groverOracle(register):

      def oracle(a):
        bit = 0
        if a == target:
          bit = 1
        return bit
		
      for i in range(register.n_states_):
        register.state_[i] *= (-1) ** oracle(i)
      return register.state_
    
    startTime = time.process_time()
    result = Grovers(groverOracle, groverList, register)
    result.groversAlgorithm()
    timeElapsed = time.process_time() - startTime
    return timeElapsed

In [ ]:
# code to time how long Grover's algorithm takes for different numbers of qubits

qubitMin = 2
qubitMax = 18
timeLimit = 600



groveraFits = []


for j in range(simRange):
    qubitListGrovers = list(range(qubitMin,qubitMax+1))
    timeElapsedGrovers = list(range(qubitMin,qubitMax+1))
    for i in range(len(qubitListGrovers)):
        n_qubits = qubitListGrovers[i]
        print(n_qubits,j,"QUBIT INIT GROVERS")
        timeElapsedGrovers[i] = groversTimeElapsed(n_qubits)
        print("Time elapsed",timeElapsedGrovers[i])
        print()
    aFit = scipy.optimize.curve_fit(fitFunc,qubitListGrovers,timeElapsedGrovers)
    groveraFits.append(aFit[0][0])
nonNeg = 0

print(np.average(groveraFits))
print(np.var(groveraFits))



In [ ]:
# code to time how long a simple Grover's algorithm takes for different numbers of qubits

qubitMin = 2
qubitMax = 19

simpleGroveraFits = []
for j in range(simRange):
    qubitListSimpleGrovers = list(range(qubitMin,qubitMax+1))
    timeElapsedSimpleGrovers = list(range(qubitMin,qubitMax+1)) 
    for i in qubitListSimpleGrovers:
        t1 = time.time()
        rand_list = np.arange(2**i)
        target = np.random.randint(2**i - 1)
        def oracle(register):
            register.state_[target] *= -1
            return register.state_
        reg = QuantumRegister(int(i))
        grover = Grovers(oracle, rand_list, reg)
        grover.groversAlgorithm()
        t2 = time.time()
        timeElapsedSimpleGrovers.append(t2-t1)
    aFit = scipy.optimize.curve_fit(fitFunc,qubitListSimpleGrovers,timeElapsedSimpleGrovers)
    simpleGroveraFits.append(aFit[0][0])

print(np.average(simpleGroveraFits))
print(np.var(simpleGroveraFits))




In [8]:
# Code to time how long Deutsch's algorithm takes  

# balanced oracle function
def balanced(register):
    def oracle(a):
	    return a%2
    for i in range(register.n_states_):
	    register.state_[i] *= (-1) ** oracle(i)
    return register.state_

#constant oracle function
def constant(register):
	def oracle(a):
		return 0
	for i in range(register.n_states_):
		register.state_[i] *= (-1) ** oracle(i)
	return register.state_


def deutschTimeElapsed():
	register = QuantumRegister(2)

	deutschBalanced = Deutsch(balanced, register)
	timeInit = time.time()
	is_balanced_one = deutschBalanced.deutschAlgorithm()
	timeBalanced = time.time() - timeInit

	register = QuantumRegister(2)
	
	deutschConstant = Deutsch(constant, register)
	timeInit = time.time()
	is_balanced_two = deutschConstant.deutschAlgorithm()
	timeConstant = time.time() - timeInit

	return timeBalanced + timeConstant
	

	
times = np.zeros(10000) 
for i in range(len(times)):
	times[i] = deutschTimeElapsed()

print(f"Average time: {np.average(times)}\nVariance: {np.var(times)}")


Average time: 0.0008369198799133301
Variance: 2.1790199386640784e-07


In [ ]:
# code to time how long the Deutsch-Josza algorithm takes for specified numbers of qubits

# balanced oracle function
def balanced(register):
	
    def oracle(a):
	    return a%2
	
	
    for i in range(register.n_states_):
	    register.state_[i] *= (-1) ** oracle(i)
    
    return register.state_

#constant oracle function
def constant(register):
	def oracle(a):
		return 0
	for i in range(register.n_states_):
		register.state_[i] *= (-1) ** oracle(i)
	return register.state_

def dJTimeElapsed(n_qubits):
	# This function is used to run the deutsch algorithm, it uses examples for both balanced and constant functions
	# The deutsch algorithm is used to evaluate a black box algorithm using a basic quantum circuit, for all constant 
	# functions our implementation will produce a 0 and for all balanced, a 1

	register = QuantumRegister(n_qubits)

	dJBalanced = DeutschJozsa(balanced,register)
	timeInit = time.time()
	is_balanced_one = dJBalanced.deutschJozsaAlgorithm()
	timeBalanced = time.time() - timeInit

	register = QuantumRegister(n_qubits)
	
	dJConstant = DeutschJozsa(constant,register)
	timeInit = time.time()
	is_balanced_two = dJConstant.deutschJozsaAlgorithm()
	timeConstant = time.time() - timeInit

	return timeBalanced + timeConstant


In [ ]:
# code to time how long the Deutsch-Josza algorithm takes for different numbers of qubits

qubitMin = 2
qubitMax = 20

DJaFits = []
for j in range(simRange):
    qubitListDJ = list(range(qubitMin,qubitMax+1))
    timeElapsedDJ = list(range(qubitMin,qubitMax+1))
    for i in range(len(qubitListDJ)):
        n_qubits = qubitListDJ[i]
        print(n_qubits,"QUBIT INIT DJ")
        timeElapsedDJ[i] = dJTimeElapsed(n_qubits)
        print("Time elapsed",timeElapsedDJ[i])
        print()
    aFit = scipy.optimize.curve_fit(fitFunc,qubitListDJ,timeElapsedDJ)
    DJaFits.append(aFit[0][0])
print(np.average(DJaFits))
print(np.var(DJaFits))


In [ ]:
# code to get memory usage for the register initialised with a specified number of qubits

def registerBitSize(n_qubits):
    register = QuantumRegister(n_qubits)
    return sys.getsizeof(register.state_)

qubitMin = 1
qubitMax = 30

qubitListMemory = list(range(qubitMin,qubitMax+1))
qubitMemorySize = list(range(qubitMin,qubitMax+1))

for i in range(len(qubitMemorySize)):
    n_qubits = qubitListMemory[i]
    qubitMemorySize[i] = registerBitSize(n_qubits)



In [ ]:
#Example grovers

yfit = scipy.optimize.curve_fit(fitFunc,qubitListGrovers,timeElapsedGrovers)
print(yfit[0])
detailedX = np.linspace(qubitListGrovers[0],qubitListGrovers[-1],100)
newY = fitFunc(detailedX,yfit[0][0],yfit[0][1])
plt.scatter(qubitListGrovers, timeElapsedGrovers)
plt.plot(detailedX, newY)
plt.xlabel("Number of Qubits")
plt.ylabel("Time taken for a Grovers Iteration")

plt.title("Time For Grovers to run")
plt.show()


In [ ]:
# Example Simple Grovers
yfit = scipy.optimize.curve_fit(fitFunc,qubitListSimpleGrovers,timeElapsedSimpleGrovers)
print(yfit[0])
detailedX = np.linspace(qubitListSimpleGrovers[0],qubitListSimpleGrovers[-1],100)
newY = fitFunc(detailedX,yfit[0][0],yfit[0][1])
plt.scatter(qubitListSimpleGrovers, timeElapsedSimpleGrovers)
plt.plot(detailedX, newY)
plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(2))
plt.xlabel("Number of Qubits")
plt.ylabel("Time taken for a Grovers Iteration")
plt.title("Time For simple Grovers to run")
plt.show()

In [ ]:
# Example DJ

yfit = scipy.optimize.curve_fit(fitFunc,qubitListDJ,timeElapsedDJ)
print(yfit[0])
detailedX = np.linspace(qubitListDJ[0],qubitListDJ[-1],100)
newY = fitFunc(detailedX,yfit[0][0],yfit[0][1])
plt.scatter(qubitListDJ, timeElapsedDJ)
plt.plot(detailedX, newY)
plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(2))
plt.xlabel("Number of Qubits")
plt.ylabel("Time taken for a DJ Iteration")
plt.title("Time For DJ to run")
plt.show()

In [ ]:
# Memory fit

yfit = scipy.optimize.curve_fit(fitFunc,qubitListMemory[nonNeg:],qubitMemorySize[nonNeg:])
print(yfit[0])
detailedX = np.linspace(qubitListMemory[0],qubitListMemory[-1],100)
newY = fitFunc(detailedX,yfit[0][0],yfit[0][1])
plt.scatter(qubitListMemory, qubitMemorySize)
plt.plot(detailedX, newY)
plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(2))
plt.xlabel("Number of Qubits")
plt.ylabel("Bits of data used")
plt.title("Register memory over Qubits")
plt.show()